<a href="https://colab.research.google.com/github/evanspap/colab/blob/main/number_theory/Euler_totient_prime_factorization_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prime Factorization — Step by Step (Trial Division)

This notebook provides an **educational, traceable** implementation of prime factorization.

It includes:

- A **step-by-step** factorization routine (verbose tracing)
- A **prime-only candidate** optimization (build primes incrementally; test only primes as divisors)
- Sorted/pretty output formatting
- A simple logical flow diagram

> Notes:
> - Python `int` supports **arbitrary precision** (no overflow like `numpy.int64`).
> - Trial division is not optimal for very large integers (e.g., 20+ digits), but is excellent for learning and small/medium inputs.


## Logical Flow Diagram

```
START
 │
 │  n given
 │
 ├── Handle factor 2 separately
 │
 ├── p = 3
 │
 ├── WHILE p² ≤ n
 │      │
 │      ├── (prime-only version) accept p only if p is prime
 │      │
 │      ├── IF n % p == 0
 │      │      │
 │      │      ├── WHILE n % p == 0
 │      │      │      ├── count p
 │      │      │      ├── n = n / p
 │      │      │      └── record factor p
 │      │      │
 │      │      └── p fully removed
 │      │
 │      └── p = p + 2
 │
 ├── IF n > 1
 │      └── n is prime → record it
 │
 └── END
```


In [1]:
def pretty_factorization(n_original: int, factors: dict) -> str:
    """Return a human-readable factorization string: n = 2^a × 3^b × ..."""
    parts = []
    for p in sorted(factors):
        exp = factors[p]
        parts.append(f"{p}" if exp == 1 else f"{p}^{exp}")
    return f"{n_original} = " + " × ".join(parts)


In [2]:
def is_prime_by_primes(x: int, primes: list[int]) -> bool:
    """
    Check if x is prime using ONLY the primes already found.
    Tests divisibility by primes q up to q*q <= x.
    """
    for q in primes:
        if q * q > x:
            break
        if x % q == 0:
            return False
    return True


In [3]:
def factorize_prime_only(n: int, verbose: bool = True):
    """
    Factorization by trial division using only primes as candidate divisors.
    Builds a prime list incrementally.

    Returns:
      (original_n, factors_dict, primes_tested, found_primes, prime_list_built)
    """
    if n <= 1:
        raise ValueError("n must be > 1")

    original_n = n
    factors = {}          # {prime: exponent}
    primes_tested = []    # primes that were actually used as candidate divisors
    found_primes = []     # prime factors found (with multiplicity, in extraction order)

    # Confirmed primes used for primality testing of candidates
    prime_list = [2]

    # --- Handle factor 2 separately ---
    if verbose:
        print("➡ Checking factor 2")

    while n % 2 == 0:
        if verbose:
            print(f"  {n} ÷ 2 = {n//2}")
        factors[2] = factors.get(2, 0) + 1
        n //= 2
        found_primes.append(2)

    if verbose:
        print(f"  → Reduced n = {n}")
        print("-" * 60)

    # --- Now test only prime odd candidates (3, 5, 7, 11, 13, ...) ---
    p = 3
    if verbose:
        print("➡ Checking only prime divisors up to √n (prime-only candidates)\n")

    # Stop when p^2 > n (no smaller divisor can remain)
    while p * p <= n:

        # Decide if candidate p is prime, using the primes already known.
        # If p is composite, skip it (do not use it as a divisor for n).
        if is_prime_by_primes(p, prime_list):
            prime_list.append(p)
            primes_tested.append(p)

            if verbose:
                print(f"✅ Candidate divisor accepted (prime): p = {p}")

            if n % p == 0:
                if verbose:
                    print(f"✔ {p} divides n")

                # Remove all copies of p from n, counting the exponent
                while n % p == 0:
                    if verbose:
                        print(f"  {n} ÷ {p} = {n//p}")
                    factors[p] = factors.get(p, 0) + 1
                    n //= p
                    found_primes.append(p)

                if verbose:
                    print(f"  → Total power: {p}^{factors[p]}")
                    print(f"  → Reduced n = {n}\n")
            else:
                if verbose:
                    print(f"✘ {p} does not divide n\n")

        #else:
            #if verbose:
                #print(f"⏭ Skipping composite candidate p = {p}")

        p += 2

    # If anything remains > 1, it must be prime.
    if n > 1:
        if verbose:
            print(f"➡ Remaining number {n} is prime; recording it.")
        factors[n] = factors.get(n, 0) + 1
        found_primes.append(n)

    return original_n, factors, primes_tested, found_primes, prime_list

## Run

Enter an integer `n > 1`.  
The notebook will trace each step and then print a sorted, “pretty” final result.


In [4]:
n = int(input("Enter an integer n (>1): "))
original_n, factors, primes_tested, found_primes, prime_list = factorize_prime_only(n, verbose=False)

Enter an integer n (>1): 562983745036825


In [5]:
print("-" * 60)
print("Factors (sorted):")
for p in sorted(factors):
    print(f"  {p}^{factors[p]}")
print(factors)
print("\nPretty form:")
print(pretty_factorization(original_n, factors))

print("\nPrimes used as candidate divisors (primes_tested):")
print(primes_tested)

print("\nPrime factors found (with multiplicity) (found_primes):")
print(found_primes)

print("\nPrime list built during the run (prime_list):")
print(prime_list)


------------------------------------------------------------
Factors (sorted):
  5^2
  7^2
  47^1
  5569^1
  1755839^1
{5: 2, 7: 2, 47: 1, 5569: 1, 1755839: 1}

Pretty form:
562983745036825 = 5^2 × 7^2 × 47 × 5569 × 1755839

Primes used as candidate divisors (primes_tested):
[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 

### Euler's Totient Function φ(n)

Once the prime factorization of an integer `n` is known,

\[
  n = \prod_i p_i^{a_i},
\]

Euler's totient function can be computed using the formula:

\[
\varphi(n) = \prod_i p_i^{a_i - 1}(p_i - 1).
\]

This follows from the multiplicative property of the totient
function and avoids any need for brute-force counting or
floating-point arithmetic.

The computation below applies this formula directly using the
previously computed prime factorization.


### Euler's Totient Function φ(n)

If the prime factorization of n is:

\[
n = \prod_i p_i^{a_i}
\]

then Euler's totient function is given by:

\[
\varphi(n) = \prod_i p_i^{a_i - 1}(p_i - 1)
\]


# ------------------------------------------------------------
# Euler's Totient Function φ(n)
#
# If the prime factorization of n is:
#     n = ∏ p_i^a_i
#
# then Euler's totient function is given by:
#     φ(n) = ∏ p_i^(a_i - 1) * (p_i - 1)
#
# This formula follows directly from the multiplicative
# property of φ(n) and allows us to compute φ(n) efficiently
# once the prime factorization is known.
#
# No floating-point arithmetic is used; all operations
# are exact integer computations.
# ------------------------------------------------------------
\

In [6]:
# --- Euler's Totient Function φ(n) using:  φ(n) = ∏ p^(a-1) * (p-1) ---
phi_n = 1
for p, a in factors.items():
    phi_n *= (p ** (a - 1)) * (p - 1)

print("\nEuler's Totient Function:")
print(f"φ({original_n}) = {phi_n}")



Euler's Totient Function:
φ(562983745036825) = 377764191221760
